In [1]:
import pandas as pd


In [2]:
pd.__version__

'1.4.2'

In [5]:
#  !pip install pyarrow

     |████████████████████████████████| 42.3 MB 34.9 MB/s eta 0:00:01     |███████████████████████▋        | 31.2 MB 34.9 MB/s eta 0:00:01


In [6]:
#  training data Deployment model - homework
df_jan = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [7]:
df_jan.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.0,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.4,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.3,1.0,N,140,186,1,18.4,3.5,0.5,4.65,0.0,1.0,28.05,2.5,0.00


In [8]:
import sklearn

In [9]:
sklearn.__version__

'1.0.2'

In [10]:
df_jan.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'Airport_fee'],
      dtype='object')

In [11]:
# Q1. Downloading the data
# We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

# Download the data for January and February 2023.

# Read the data for January. How many columns are there? 
# 16
# 17
# 18
# 19 - Correct

In [12]:
df_jan.shape[1] 

19

In [13]:
# Q2. Computing duration
# Now let's compute the duration variable. It should contain the duration of a ride in minutes.

# What's the standard deviation of the trips duration in January?

# 32.59
# 42.59 - correct
# 52.59
# 62.59

In [14]:
df_jan['duration_in_minutes'] = (df_jan['tpep_dropoff_datetime'] - df_jan['tpep_pickup_datetime']).dt.total_seconds()/60

In [15]:
df_jan.head(2)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration_in_minutes
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.0,1.0,N,238,42,2,8.6,1.0,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.4,1.0,N,138,231,1,52.7,6.0,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333


In [18]:
print(df_jan['duration_in_minutes'].std())

43.79457870013612


In [17]:
# after rounding minutes to 2 digit
df_jan['duration_in_minutes'].round(2).std()

43.7945790323912

In [33]:
# Q3. Dropping outliers
# Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

# What fraction of the records left after you dropped the outliers?

# 90%
# 92%
# 95%
# 98% - correct

In [19]:
total_rows = len(df_jan)
df_jan = df_jan[(df_jan['duration_in_minutes']>=1) & (df_jan['duration_in_minutes']<=60)]
filtered_rows_count = len(df_jan)
fraction = round((filtered_rows_count/total_rows)*100, 2)
print(fraction)

97.43


In [35]:
# Q4. One-hot encoding
# Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

# Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
# Fit a dictionary vectorizer
# Get a feature matrix from it
# What's the dimensionality of this matrix (number of columns)?

# 2
# 155
# 345
# 515 --- Correct
# 715

In [20]:
from sklearn.feature_extraction import DictVectorizer

# print(df_jan.head(2))
features = df_jan[['PULocationID','DOLocationID']].astype(str)
dictionary = features.to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(dictionary)
print(X_train.shape)

(3316216, 518)


In [37]:
# Q5. Training a model
# Now let's use the feature matrix from the previous step to train a model.

# Train a plain linear regression model with default parameters, where duration is the response variable
# Calculate the RMSE of the model on the training data
# What's the RMSE on train?

# 3.64
# 7.64 -correct
# 11.64
# 16.64

In [23]:
from sklearn.linear_model import LinearRegression
# from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import mean_squared_error
import numpy as np
y_train = df_jan['duration_in_minutes']

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_train)

# rsme = mean_squared_error(y_train, y_pred, squared=False) # Since this is old version, other rsme is present

# print(rsme)

In [24]:
print(y_pred.std())

6.7745437754354985


In [ ]:
# Q6. Evaluating the model
# Now let's apply this model to the validation dataset (February 2023).

# What's the RMSE on validation?

# 3.81
# 7.81  - correct
# 11.81
# 16.81

In [40]:
# Test Data
df_feb = pd.read_parquet('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet')
df_feb.head()


,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2.0,0.30,1.0,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1.0,0.00,1.0,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0.0,18.80,1.0,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1.0,3.22,1.0,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00


In [41]:
df_feb['duration_in_minutes'] = (df_feb['tpep_dropoff_datetime'] - df_feb['tpep_pickup_datetime']).dt.total_seconds()/60
df_feb = df_feb[(df_feb['duration_in_minutes']>=1) & (df_feb['duration_in_minutes']<=60)]
df_feb.shape[0]


features_feb = df_feb[['PULocationID','DOLocationID']].astype(str)
dictionary = features_feb.to_dict(orient='records')
X_test = dv.transform(dictionary)
print(X_test.shape)

(2855951, 515)


In [42]:
y_test = df_feb['duration_in_minutes']
# Predict test data
y_pred = model.predict(X_test)

rsme = mean_squared_error(y_test, y_pred, squared=False)
print(rsme)

7.811832641626525
